# Major Model Analysis Ahead

In [1]:
from utils.model_analysis import * 
import os
import matplotlib.pyplot as plt
#from nilearn import datasets, plotting

In [2]:
import torch
#import torchvision.models as models

# Instantiate the model
model = torch.load('/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/pretrained_alexnet.pt')

# Create a dummy input
dummy_input = torch.randn(1, 3, 224, 224)

# Pass the dummy input through the model up to the layer of interest
layer_of_interest = 6  # Change this to the index of your layer of interest
model = torch.nn.Sequential(*(list(model.features.children())[:layer_of_interest + 1]))
output = model(dummy_input)
output_shape = output.shape[1] * output.shape[2] * output.shape[3]

# Print the output shape
print(output.shape)

torch.Size([1, 384, 13, 13])


In [3]:
if __name__ == "__main__":
    #Change as needed
    checkpoint_path = '/Users/emilykruger/Documents/GitHub/aml_project_2023/hpc/utils/trained_models/alexnet_LR0.00015_SAMPLES_200_EPOCHS100_BATCHSIZE_16_TIME_2023-12-07_23:18:19.pt'
    output_size = 100
    backbone = 'alexnet'
    #load images and original activations
    subject_id = 1
    brain, image_paths = load_subject_data(1, index_start=0, index_end=5)

    #pick image and activation
    image = np.load(image_paths[0])
    image = preprocess(image)

    activation = torch.Tensor(brain[0,:])
    print('activation type:', type(activation))

    scores = {}

    # Check if GPU is available and if not, use CPU
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    #Load in model & Create feature extractor
    trained_model_state_dict = torch.load(checkpoint_path, map_location = device)
    feature_extractor = torch.hub.load('utils', backbone, source = 'local')

    trained_model = ResNet1HeadID(output_size = output_size, feature_extractor= feature_extractor).eval()
    trained_model.load_state_dict(trained_model_state_dict["model_state_dict"])
    trained_model_backbone = trained_model.feature_extractor

    if backbone == "alexnet":
        output_layer_names = get_module_names(trained_model_backbone)[1:-1]
    else:
        output_layer_names = get_block_names(trained_model_backbone)[:-1]

    feature_extractor = create_feature_extractor(trained_model_backbone, output_layer_names)

    #Extract features & predict fMRI data 
    outputs = feature_extractor(image.unsqueeze(0))
    flat_outputs = flatten_features(outputs)
    predictions = make_prediction(trained_model, flat_outputs, output_shape, subject = subject_id)
    print(predictions)

    #Inverse transform of preds with frozen PCA models
    pca = get_pca_model(subject_id)
    inversed_predictions = predictions.copy()
    for key in inversed_predictions:
        #convert prediction tensors to np arrays to make it compatible for inverse pca
        preds = inversed_predictions[key].detach().numpy()
        preds = torch.Tensor(pca.inverse_transform(preds))
        print('preds type:', type(preds))
        # inverse-pca and store in new dict
        inversed_predictions[key] = preds

   #Caculating MNNPC on Preds
    # for key in inversed_predictions:
    #     preds = inversed_predictions[key]
    #     mnnpc = MNNPC()
    #     score = mnnpc(pred = preds, gt=activation)
    #     scores[key] = score

    # print(scores)

Shape of pca_brain:  (5, 39548)
activation type: <class 'torch.Tensor'>


/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/Users/emilykruger/Documents/GitHub/aml_project_2023/aml_project_2023/aml_venv/lib/python3.11/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,


: 